In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import os
import esm

#Does not work :(

# Define the parameters
short_city=['Si','Ro','Mi']
long_city=['Siracusa','Roma','Milano']

for ee in range(1,4):
    for ng in range(1,4):
        scenario=f'E{ee}G{ng}'
        print(scenario)
        for c in range(3):
            city=short_city[c]
            city_extended=long_city[c]
            # Create folder if it doesn't exist
            output_folder = f'{city_extended}/Results_csv'
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            # Change the name of scenario input file into input_data.xlsx
            specific_excel_file = f'{city_extended}/input_data/input_data_{scenario}.xlsx'
            input_excel_file = f'{city_extended}/input_data/input_data.xlsx'
            if os.path.exists(specific_excel_file):
                os.rename(specific_excel_file, input_excel_file)
            else:
                print(f'File {specific_excel_file} does not exist')
            
            #Run the model
            main_dir_path = ''

            model = esm.Model(
                model_dir_name=city_extended,
                main_dir_path=main_dir_path,
                use_existing_data=True,
                log_level='info',
            )

            model.update_database_and_problem(force_overwrite=True)

            model.run_model(verbose=False, solver='GUROBI')

            model.load_results_to_database(main_dir_path,model_dir_name)

            #Rename folder
            csv_folder = f'{city_extended}/{scenario}_Results_csv'
            if os.path.exists(output_folder):
                os.rename(output_folder, csv_folder)
            else:
                print('No results folder found')

            cost_op_tot_all = pd.read_csv(f'{csv_folder}/cost_op_tot.csv',usecols=lambda column: column != 'id')
            active_techs_all = pd.read_csv(f'{csv_folder}/active_techs.csv',usecols=lambda column: column != 'id')
            cap_tot_all = pd.read_csv(f'{csv_folder}/cap_tot.csv',usecols=lambda column: column != 'id')
            new_units_all = pd.read_csv(f'{csv_folder}/new_units.csv',usecols=lambda column: column != 'id')
            cap_new_all = pd.read_csv(f'{csv_folder}/cap_new.csv',usecols=lambda column: column != 'id')
            CO2_tot_all = pd.read_csv(f'{csv_folder}/CO2_tot.csv',usecols=lambda column: column != 'id')
            CO2_act_all = pd.read_csv(f'{csv_folder}/CO2_act.csv',usecols=lambda column: column != 'id')
            insulation_all = pd.read_csv(f'{csv_folder}/insulation.csv',usecols=lambda column: column != 'id')
            X_all = pd.read_csv(f'{csv_folder}/X.csv',usecols=lambda column: column != 'id')
            Xt_all = pd.read_csv(f'{csv_folder}/X_t.csv',usecols=lambda column: column != 'id')
            Q_all = pd.read_csv(f'{csv_folder}/Q.csv',usecols=lambda column: column != 'id')
            SOC_all = pd.read_csv(f'{csv_folder}/SOC.csv',usecols=lambda column: column != 'id')
            # Importing exogenous data from data_input excel
            excel_file = f'{city_extended}/input_data/input_data.xlsx'
            Y_all = pd.read_excel(excel_file, sheet_name='Y',usecols=lambda column: column != 'id')
            cost_cap = pd.read_excel(excel_file, sheet_name='cost_cap',usecols=lambda column: column != 'id')
            cost_period = pd.read_excel(excel_file, sheet_name='cost_period',usecols=lambda column: column != 'id')
            cost_inv_all = pd.read_excel(excel_file, sheet_name='cost_inv',usecols=lambda column: column != 'id')
            disc_frac_pd = pd.read_excel(excel_file, sheet_name='disc_frac',usecols=lambda column: column != 'id')
            disc_frac=disc_frac_pd.at[0,'values']
            TI_cost_all = pd.read_excel(excel_file, sheet_name='TI_cost',usecols=lambda column: column != 'id')
            DPT = pd.read_excel(excel_file, sheet_name='days_per_type',usecols=lambda column: column != 'id')
            

            for user in ['A','B','C','D']:
                user_name=f'User{user}'
                scenario_name=f'{scenario}_{city}_{user}'
                #Selecting for specific user (letter)
                cost_op_tot=cost_op_tot_all.loc[cost_op_tot_all['s_names'] == f'User{user}']
                active_techs=active_techs_all.loc[active_techs_all['s_names'] == f'User{user}']
                cap_tot=cap_tot_all.loc[cap_tot_all['s_names'] == f'User{user}']
                new_units=new_units_all.loc[new_units_all['s_names'] == f'User{user}']
                cap_new=cap_new_all.loc[cap_new_all['s_names'] == f'User{user}']
                CO2_tot=CO2_tot_all.loc[CO2_tot_all['s_names'] == f'User{user}']
                CO2_act=CO2_act_all.loc[CO2_act_all['s_names'] == f'User{user}']
                insulation=insulation_all.loc[insulation_all['s_names'] == f'User{user}']
                X=X_all.loc[X_all['s_names'] == f'User{user}']
                Xt=Xt_all.loc[Xt_all['s_names'] == f'User{user}']
                Q=Q_all.loc[Q_all['s_names'] == f'User{user}']
                SOC=SOC_all.loc[SOC_all['s_names'] == f'User{user}']
                Y=Y_all.loc[Y_all['s_names'] == f'User{user}']
                cost_inv=cost_inv_all.loc[cost_inv_all['s_names'] == f'User{user}']
                TI_cost=TI_cost_all.loc[TI_cost_all['s_names'] == f'User{user}']

                for df in [cost_op_tot, active_techs, cap_tot, new_units, cap_new, CO2_tot, CO2_act, insulation, X, Xt, Q, SOC, Y, cost_inv, TI_cost]:
                    df.reset_index(drop=True, inplace=True)

                #Definitions for time
                seasons = ['wi', 'mc', 'mw', 'su']
                season_names = {'wi': 'Winter', 'mc': 'Mid-cold', 'mw': 'Mid-warm', 'su': 'Summer'}

                #Isolate peak
                peak_Y= Y[Y['th_names'] == 'peak']
                Y=Y[Y['th_names'] != 'peak']
                peak_Q= Q[Q['th_names'] == 'peak']
                Q=Q[Q['th_names'] != 'peak']
                peak_X=X[X['th_names'] == 'peak']
                X=X[X['th_names'] != 'peak']
                peak_Xt=Xt[Xt['th_names'] == 'peak']
                Xt=Xt[Xt['th_names'] != 'peak']
                SOC=SOC[SOC['th_names'] != 'peak']
                DPT=DPT[DPT['days_names'] != 'peak']

                for db in [Y,Q,X,Xt,SOC]:
                    db.index=range(len(db))
                    db.loc[:,'season']=db['th_names'].str[4:6]
                    db['hour'] = db['th_names'].str[1:3].astype(int)
                    db['day'] = db['th_names'].str[-2:].astype(int)
                    db['time'] = (db['day'] - 1) * 24 + db['hour']

                #Defining values to export in results.xlsx
                activity_costs=sum(cost_op_tot['values'])
                periodic_costs=sum((cost_period['values']+(cost_cap['values']*cap_tot['values']))*active_techs['values'])
                bill_one_year=activity_costs+periodic_costs
                operation_costs_discounted=bill_one_year*(sum(disc_frac ** x for x in range(1, 11)))
                investment_costs=sum(cost_inv['values']*new_units['values'])
                NPC=operation_costs_discounted+investment_costs

                cost_EE_period=cost_period.loc[cost_period['t_names']=='National Grid EE']['values'].values[0]+cost_cap.loc[cost_cap['t_names']=='National Grid EE']['values'].values[0]*cap_tot.loc[cap_tot['t_names']=='National Grid EE']['values'].values[0]
                cost_EE=cost_op_tot.loc[cost_op_tot['a_names']=='National Grid import']['values'].values[0]+cost_op_tot.loc[cost_op_tot['a_names']=='National Grid export']['values'].values[0]+cost_EE_period

                cost_NG_period=active_techs[active_techs['t_names']=='Gas National Grid']['values'].values[0]*cost_period.loc[cost_period['t_names']=='Gas National Grid']['values'].values[0]+cost_cap.loc[cost_cap['t_names']=='Gas National Grid']['values'].values[0]*cap_tot.loc[cap_tot['t_names']=='Gas National Grid']['values'].values[0]
                cost_NG=cost_op_tot.loc[cost_op_tot['a_names']=='Natural Gas supply grid']['values'].values[0]+cost_NG_period

                def compute_daily(df):
                    #Group by day and season
                    df = df.groupby(['day', 'season']).agg(consumption=('values', 'sum')).reset_index()
                    #Fix the order
                    season_order = ['wi', 'mc', 'mw', 'su']
                    df['season'] = pd.Categorical(df['season'], categories=season_order, ordered=True)
                    df_sorted = df.sort_values(by=['season', 'day'])
                    df_sorted.index = range(len(df_sorted))
                    return df_sorted

                Q_EE=Q[Q['n_names']=='EE']
                Q_EE_daily = compute_daily(Q_EE)
                Q_EE_daily['consumption'] = Q_EE_daily['consumption']*DPT['values']
                Q_EE_tot=sum(Q_EE_daily['consumption'])

                X_EE_daily = compute_daily(X[X['a_names']=='National Grid import'])
                X_EE_daily['consumption'] = X_EE_daily['consumption']*DPT['values']

                PV_prod=X[X['a_names']=='PV']
                X_PV_daily = compute_daily(PV_prod)
                X_PV_daily['consumption'] = X_PV_daily['consumption']*DPT['values']

                PV_sell=X[X['a_names']=='National Grid export']
                X_sell_daily = compute_daily(PV_sell)
                X_sell_daily['consumption'] = X_sell_daily['consumption']*DPT['values']

                Q_NG_daily = compute_daily(Q[Q['n_names']=='Natural Gas'])
                Q_NG_tot=sum(Q_NG_daily['consumption']*DPT['values'])

                #Exporting results for this case
                results=pd.DataFrame(columns=['city','size','NPC','opex','capex','PV','GridEE','StorageEE','GasBoiler','HP','AC','StorageHW','Radiators','Floor_heat','Induction','GasStove','Q_EE','Q_NG','EE_in','EE_out','EE_PV','cash_EEin','cash_EEout','cash_EEtot','cash_NGin','cash_NGtot','CO2'])

                results.at[scenario_name,'city']=city_extended
                results.at[scenario_name,'size']=user
                results.at[scenario_name,'NPC']=round(NPC,2)
                results.at[scenario_name,'opex']=round(bill_one_year,2)
                results.at[scenario_name,'capex']=round(investment_costs,2)
                results.at[scenario_name,'PV']=round(cap_new.loc[cap_new['t_names']=='PV sys']['values'].values[0],2)
                results.at[scenario_name,'GridEE']=round(cap_tot.loc[cap_tot['t_names']=='National Grid EE']['values'].values[0],2)
                results.at[scenario_name,'StorageEE']=round(cap_tot.loc[cap_tot['t_names']=='Storage EE']['values'].values[0],2)
                results.at[scenario_name,'GasBoiler']=round(cap_tot.loc[cap_tot['t_names']=='Gas Boiler']['values'].values[0],2)
                results.at[scenario_name,'HP']=round(cap_tot.loc[cap_tot['t_names']=='HP']['values'].values[0],2)
                results.at[scenario_name,'AC']=round(cap_tot.loc[cap_tot['t_names']=='AC']['values'].values[0],2)
                results.at[scenario_name,'StorageHW']=round(cap_tot.loc[cap_tot['t_names']=='Storage HW']['values'].values[0],2)
                results.at[scenario_name,'Radiators']=round(cap_tot.loc[cap_tot['t_names']=='Radiators']['values'].values[0],2)
                results.at[scenario_name,'Floor_heat']=round(cap_tot.loc[cap_tot['t_names']=='Floor heating']['values'].values[0],2)
                results.at[scenario_name,'Induction']=round(cap_tot.loc[cap_tot['t_names']=='Induction stove']['values'].values[0],2)
                results.at[scenario_name,'GasStove']=round(cap_tot.loc[cap_tot['t_names']=='Gas stove']['values'].values[0],2)
                results.at[scenario_name,'Q_EE']=round(Q_EE_tot,2)
                results.at[scenario_name,'Q_NG']=round(Q_NG_tot,2)
                results.at[scenario_name,'EE_in']=round(sum(X_EE_daily['consumption']),2)
                results.at[scenario_name,'EE_out']=round(sum(X_sell_daily['consumption']),2)
                results.at[scenario_name,'EE_PV']=round(sum(X_PV_daily['consumption']),2)
                results.at[scenario_name,'cash_EEin']=round(cost_op_tot.loc[cost_op_tot['a_names']=='National Grid import']['values'].values[0],2)
                results.at[scenario_name,'cash_EEout']=round(cost_op_tot.loc[cost_op_tot['a_names']=='National Grid export']['values'].values[0],2)
                results.at[scenario_name,'cash_EEtot']=round(cost_EE,2)
                results.at[scenario_name,'cash_NGin']=round(cost_op_tot.loc[cost_op_tot['a_names']=='Natural Gas supply grid']['values'].values[0],2)
                results.at[scenario_name,'cash_NGtot']=round(cost_NG,2)
                results.at[scenario_name,'CO2']=round(CO2_tot.iloc[0]['values'],4)

                file_path = "/Users/cami/Documents/GitHub/pyesm_thesis/tesi_camilla/casi_MarioU/results_database.xlsx"  # Sostituisci con il percorso corretto del file
                book = load_workbook(file_path)
                # Trova l'ultima riga occupata nel foglio
                sheet_name = "Sheet1"
                sheet = book[sheet_name]
                last_row = sheet.max_row

                # Scrivi i dati usando l'ExcelWriter in modalità append
                with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
                    results.to_excel(writer, sheet_name=sheet_name, index=True, header=False, startrow=last_row)

                # Change back the name of input_data into scenario input file

                if os.path.exists(input_excel_file):
                    os.rename(input_excel_file, specific_excel_file)
                else:
                    print(f'File {input_excel_file} does not exist')